# 3章. テキスト分析1：テキストのベクトル化


## 3.1 テキスト分析の目的


In [ ]:
!apt -y install fonts-ipafont-gothic
!pip install spacy[ja]


In [ ]:
# 本章で用いる各種モジュールのインポート。
import os
import urllib.request
import tarfile
import math
import re
import unicodedata
import platform
import random
import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
from spacy.lang.ja import Japanese
from wordcloud import WordCloud
from IPython.display import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties


In [ ]:
# 本章で用いる各種設定。

# データを保持しておくディレクトリ。
DATA_DIR = "data"
# livedoorニュースコーパスのURL。
LDCC_URL = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
# livedoorニュースコーパスを保存するパス。
LDCC_FILE = os.path.join(DATA_DIR, "ldcc-20140209.tar.gz")
# livedoorニュースコーパスを解凍するディレクトリ。
NEWS_DIR = os.path.join(DATA_DIR, "text")
# データフレームを保存するCSVファイル。
CSV_PATH = os.path.join(DATA_DIR, "newsdf.csv")


In [ ]:
# livedoorニュースコーパスの入手と解凍。
# データディレクトリの作成を行う。
os.makedirs(DATA_DIR, exist_ok=True)
# ファイルのダウンロードを行う。
req = urllib.request.Request(LDCC_URL)
with urllib.request.urlopen(req) as res:
    data = res.read()
    with open(LDCC_FILE, mode="wb") as f:
        f.write(data)
# ファイルの解凍を行う。
with tarfile.open(LDCC_FILE) as tar:
    tar.extractall(DATA_DIR)


In [ ]:
# livedoorニュースコーパスのトピック情報。
category_names = ["ITライフハック", "MOVIE ENTER",
    "Peachy", "Sports Watch", "livedoor HOMME",
    "エスマックス", "トピックニュース",
    "家電チャンネル", "独女通信"]

# livedoorニュースコーパスのトピックに対応するディレクトリ名。
category_dirs = ["it-life-hack", "movie-enter",
    "peachy", "sports-watch", "livedoor-homme",
    "smax", "topic-news",
    "kaden-channel", "dokujo-tsushin"]


In [ ]:
# 全ての記事のテキストファイルを読み込む。
docs = []
for cat in category_dirs:
    pattern = os.path.join(NEWS_DIR, cat, f"{cat}*.txt")
    for src_file in sorted(glob.glob(pattern)):
        with open(src_file, "r", encoding="utf8") as f:
            url = f.readline().strip()
            date = f.readline().strip()
            title = f.readline().strip()
            body = f.read().strip()
        docs.append((cat, url, date, title, body))


In [ ]:
# 総記事数を表示する。
num_docs = len(docs)
print(f"総記事数：{num_docs}件")


In [ ]:
# DataFrameに格納して最初の5件の文書を表示する。
df = pd.DataFrame(docs, columns=["category", "url", "date",
                                 "title", "body"])
display(df.head())


In [ ]:
# インデックス0の文書のタイトルを表示。
print(df.iloc[0]["title"])


In [ ]:
# インデックス0の文書の本文を表示。
print(df.iloc[0]["body"])


## 3.2 文書の前処理


In [ ]:
# 形態素解析の準備とその利用。

# 形態素解析器の準備。
nlp = Japanese()
# インデックス0の文書のタイトルを形態素解析する。
tokens = nlp(df.iloc[0]["title"])
print("トークン番号, 表層形, 品詞, 品詞細分類, 原型")
for token in tokens:
    print(f"{token.i:>2}, {token.orth_}, {token.pos_}, {token.tag_}, {token.lemma_}")


In [ ]:
# 関数tokenizeの定義。
def tokenize(text):
    tokens = nlp(text)
    result = []
    for token in tokens:
        result.append(token.orth_)
    return " ".join(result)


In [ ]:
# 関数tokenizeの利用例。
print(tokenize("すもももももももものうち。"))


In [ ]:
# 関数get_font_pathの定義の例。インストールされているフォントファイルを指定する必要がある。
def get_font_path():
    # OSによってフォントを使い分ける。
    pf = platform.system()
    if pf == "Windows":
        return r"C:\Windows\Fonts\meiryo.ttc"
    elif pf == "Darwin":
        return "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"
    elif pf == "Linux":
        return "/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf"
    else:
        raise RuntimeError("対応していません。")


In [ ]:
# 関数show_wordcloudの定義。
def show_wordcloud(words):
    # 日本語フォントを取得する。
    font_path = get_font_path()
    # 分割テキストからwordcloudを生成する。
    wordcloud = WordCloud(font_path=font_path, background_color="white")
    wordcloud.generate(words)
    # 表示する。
    wordcloud.to_file(f"./wordcloud.png")
    display(Image(f"./wordcloud.png"))


In [ ]:
# インデックス0の文書を分かち書きして最初の100語を表示する。
words = tokenize(df.iloc[0]["body"])
print(words[0:100])


In [ ]:
# インデックス0の文書の全ての語によるワードクラウドを表示する。
show_wordcloud(words)


In [ ]:
# テキストのクリーニングを行う関数text_cleaningの定義。

re_url = re.compile(r"https?://[;/\?:@&=\+\$,0-9A-Za-z\-_\.!~\*\'\(\)%#]+")
re_num = re.compile(r"\d([\d.,]?\d)*")

def text_cleaning(text):
    # 互換等価性変換を実行する
    text = unicodedata.normalize("NFKC", text)
    # URLを削除する。
    text = re_url.sub("", text)
    # 全ての数値を0にする。
    text = re_num.sub("0", text)
    # 英語を全て小文字にする場合は、次行のコメントを外す。
    #text = text.lower()
    return text


In [ ]:
# テキストのクリーニングの実行。
text = text_cleaning(df.iloc[0]["body"])
print(text)


In [ ]:
# 関数extract_wordsの定義。
def extract_words(text):
    # テキストクリーニングを行った結果を形態素解析する。
    tokens = nlp(text_cleaning(text))
    result = []
    target_pos_set = set(("NOUN", "PROPN", "VERB", "ADJ", "ADV"))
    for token in tokens:
        # 対象となる品詞かどうかをチェックする。
        if token.pos_ in target_pos_set:
            # 原型を取得する。
            result.append(token.lemma_)
    return " ".join(result)


In [ ]:
# インデックス0の文書の名詞や動詞などのみを対象としてワードクラウドを表示する。
words = extract_words(df.iloc[0]["body"])
show_wordcloud(words)


In [ ]:
# df["words"]に分かち書きを追加する。
df = df.assign(words=(df["title"] + df["body"]).apply(text_cleaning).apply(extract_words))
# CSVファイルに保存する。
df.to_csv(CSV_PATH)


In [ ]:
# 更新されたデータフレームの表示。
display(df.head())


## 3.3 語の出現頻度を基にしたテキストのベクトル化と類似度計算


In [ ]:
# 語の出現頻度による特徴ベクトルの作成。

# 語の出現頻度による特徴ベクトルを作成するクラスを用意する。
vectorizer_count = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
# 全文書を特徴ベクトル化する。
vectors_count = vectorizer_count.fit_transform(df["words"])


In [ ]:
# CountVectorizerの設定の表示。
display(vectorizer_count.get_params())


In [ ]:
# 文書の特徴ベクトルの変数の概要を確認する。
display(vectors_count)


In [ ]:
# どのような語が次元になったのかを取得する。
vocabulary = vectorizer_count.get_feature_names_out().tolist()
# 語の種類数を表示する。
print(f"語の種類数：{len(vocabulary)}")
# ランダムに6語表示する。
print(random.sample(vocabulary, 6))


In [ ]:
# インデックス0の文書ではどのような語の出現回数が多いかを表示する。
doc_id = 0
elements = zip(vectors_count[doc_id].data, vectors_count[doc_id].indices)
elements = sorted(elements, reverse=True)

# 出現回数が多い語と出現回数を表示
for i in range(min(len(elements), 5)):
    print(f"文書（{doc_id}）での語「{vocabulary[elements[i][1]]}」の出現回数は{elements[i][0]}")


In [ ]:
# 語「神戸」がどの文書で出現頻度が多いかを表示する。
target_word = "神戸"
target_word_id = vocabulary.index(target_word)

# wordが1回以上出現している文書のインデックスを全て取得する。
doc_ids = [i for i, v in enumerate(vectors_count) if target_word_id in v.indices]

elements = []
for doc_id in doc_ids:
    # その文書の特徴ベクトルを取得する。ただし、粗行列表現である。
    vector = vectors_count[doc_id]
    # 粗行列表現の何番目が与えられたwordに対応しているかを求める。
    word_id = vector.indices.tolist().index(target_word_id)
    # 与えられたwordの出現頻度を取得する。
    count = vector.data[word_id]
    # 並べ替える要素に追加する。
    elements.append((count, doc_id))

# 出現回数が多い順に並べ替える。
elements = sorted(elements, reverse=True)

# wordの出現回数が多い文書のインデックスと出現回数を表示
for i in range(min(len(elements), 5)):
    print(f"文書（{elements[i][1]}）での語「{target_word}」の出現回数は{elements[i][0]}")


In [ ]:
# インデックス3114の文書のタイトルと本文を表示する。
print(df.iloc[3114]["title"])
print(df.iloc[3114]["body"])


In [ ]:
# 関数show_wordcloud_by_vector
def show_wordcloud_by_vector(vector, vocabulary):
    # 日本語フォントを取得する。
    font_path = get_font_path()
    # 語とその重みを表す辞書を作成する。
    vector_dict = {}
    # ベクトルは粗行列で与えられる場合とndarrayで与えられる場合を想定する。
    if not isinstance(vector, np.ndarray):
        vector = vector.toarray()[0]
    for word_id in vector.nonzero()[0]:
        vector_dict[vocabulary[word_id]] = vector[word_id]
    # 語とその重みを表す辞書からwordcloudを生成する。
    wordcloud = WordCloud(font_path=font_path, background_color="white")
    wordcloud.generate_from_frequencies(vector_dict)
    # 表示する。
    wordcloud.to_file("./wordcloud.png")
    display(Image("./wordcloud.png"))


In [ ]:
# インデックス3114の文書のワードクラウドを表示する。
show_wordcloud_by_vector(vectors_count[3114], vocabulary)


In [ ]:
# インデックス0の文書とインデックス3114の文書のコサイン類似度を計算する。
cos = cosine_similarity(vectors_count[0], vectors_count[3114])
print(cos)


In [ ]:
# インデックス3114の文書の特徴ベクトルの概要を表示する。
display(vectors_count[3114])


In [ ]:
# インデックス3114の文書の特徴ベクトルをndarrayで取得する。
v3114 = vectors_count[3114].toarray()[0]
print(v3114)


In [ ]:
v0 = vectors_count[0].toarray()[0]


In [ ]:
# インデックス0の文書とインデックス3114の文書の特徴ベクトルからコサイン類似度を計算する。
print(cosine_similarity([v0], [v3114]))


In [ ]:
# 5つの文書の全ての組み合わせについてコサイン類似度を計算する。
doc_ids = [3114, 1942, 1920, 2067, 3128]
sim_matrix = cosine_similarity(vectors_count[doc_ids],
                               vectors_count[doc_ids])
display(pd.DataFrame(sim_matrix, index=doc_ids, columns=doc_ids))


In [ ]:
# 5つの記事のタイトル（最初の25文字）を表示する。
for doc_id in doc_ids:
    print(f"{doc_id}: {df.iloc[doc_id]['title'][:25]}")


In [ ]:
# 2つの新しい文書
newdocs = [
    "今日は朝から晴れています。",
    "明日は霧雨が降るそうです。"
]


In [ ]:
# 2つの新しい文書から語の抽出を行う。
newdocs_words = list(map(extract_words, newdocs))
print(newdocs_words)


In [ ]:
# 2つの新しい文書の特徴ベクトルを取得する。
newdocs_vectors = vectorizer_count.transform(newdocs_words)
print(newdocs_vectors)


In [ ]:
# 新しい文書における語と出現頻度を表示する。
for i in range(len(newdocs)):
    vector = newdocs_vectors[i]
    for count, word_id in zip(vector.data, vector.indices):
        print(f"新しい文書（{i}）での語「{vocabulary[word_id]}」の出現頻度は{count}")


In [ ]:
# 簡易検索システムを実現する関数search_by_queryを定義する。
def search_by_query(query, vectorizer, vectors):
    print("■■■■■■■■■■■検索■■■■■■■■■■■■")
    # クエリ文字列の前処理を行い、ベクトル化する。
    query_words = extract_words(query)
    query_vector = vectorizer.transform([query_words])
    print("検索クエリ: {}".format(query_words))

    # クエリのベクトルととすべて文書のコサイン類似度を求める。
    sims = cosine_similarity(query_vector, vectors)
    # コサイン類似度と、文書インデックスのペアを作成する。
    sim_idx_pairs = zip(sims[0], range(vectors.shape[0]))
    # コサイン類似度が高い順にソートする。
    ranking_result = sorted(sim_idx_pairs, reverse=True)

    # 検索結果のトップ3件を表示する。
    print("■■■■■■■■■■検索結果■■■■■■■■■■■")
    for i in range(3):
        cos = ranking_result[i][0]
        doc_id = ranking_result[i][1]
        print(f"＝＝＝＝＝＝＝＝＝＝＝第{i+1}位＝＝＝＝＝＝＝＝＝＝＝")
        print(f"★記事ID：{doc_id}／コサイン類似度：{cos:.3f}")
        print(df.iloc[doc_id]["url"])
        print(df.iloc[doc_id]["title"][:25])


In [ ]:
# 「グルメ レストラン」で検索を行う。
search_by_query("グルメ レストラン", vectorizer_count, vectors_count)


In [ ]:
# TF-IDF重み付けによる特徴ベクトルの作成。

# TF-IDF重み付けによる特徴ベクトルを作成するクラスを用意する。
vectorizer_tfidf = TfidfVectorizer(norm=None,
                                   token_pattern=r"(?u)\b\w+\b")
# 全文書を特徴ベクトル化する。
vectors_tfidf = vectorizer_tfidf.fit_transform(df["words"])


In [ ]:
# TfidfVectorizerの設定の表示。
display(vectorizer_tfidf.get_params())


In [ ]:
# IDF値による語のヒストグラムを表示する。
plt.hist(vectorizer_tfidf.idf_, bins=11, range=(0, 11))
# 日本語フォントを取得する。
fp = FontProperties(fname=get_font_path())
plt.xlabel("IDF値", fontproperties=fp)
plt.ylabel("語数", fontproperties=fp)
plt.show()


In [ ]:
# 変数vectors_tfidfの概要を表示する。
display(vectors_tfidf)


In [ ]:
# どのような語が次元になったのかを取得する。
vocabulary = vectorizer_tfidf.get_feature_names_out().tolist()


In [ ]:
# インデックス0の文書ではどのような語のTF-IDF値が大きいかを表示する。
doc_id = 0
elements = zip(vectors_tfidf[doc_id].data, vectors_tfidf[doc_id].indices)
elements = sorted(elements, reverse=True)

# TF-IDF値が大きい語を表示
for i in range(min(len(elements), 5)):
    print(f"文書（{doc_id}）での"
          f"語「{vocabulary[elements[i][1]]}」"
          f"のTF-IDF重みは{elements[i][0]:.2f}")


In [ ]:
# TF-IDF重み付けによるワードクラウドの表示。
print("TF-IDF重み付けによるワードクラウド")
show_wordcloud_by_vector(vectors_tfidf[0], vocabulary)


In [ ]:
# 「グルメ レストラン」でTF-IDF重み付けで検索を行う。
search_by_query("グルメ レストラン", vectorizer_tfidf, vectors_tfidf)


# 4章. テキスト分析2：ベクトルを用いた分析


## 4.1 特徴ベクトルの次元圧縮とトピック抽出


In [ ]:
# 前章のものに加えて、本章で用いる各種モジュールのインポート。
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans


In [ ]:
%%time
# 潜在的意味解析で200次元に次元圧縮を行う。
lsa = TruncatedSVD(200, algorithm="arpack")
vectors_lsa = lsa.fit_transform(vectors_tfidf)


In [ ]:
# 変数vectors_lsaの行列の形を表示する。
print(vectors_lsa.shape)


In [ ]:
# インデックス0の文書の圧縮された特徴ベクトルにおける最初の3次元と最後の3次元を表示する。
print(vectors_lsa[0][:3])
print(vectors_lsa[0][-3:])


In [ ]:
# 5つの文書のコサイン類似度の計算。
doc_ids = [3114, 1942, 1920, 2067, 3128]
# TF-IDF重み付けを使った特徴ベクトルの場合。
sim_matrix_tfidf = cosine_similarity(vectors_tfidf[doc_ids], vectors_tfidf[doc_ids])
display(pd.DataFrame(sim_matrix_tfidf, index=doc_ids, columns=doc_ids))
# TF-IDF重み付けを使った特徴ベクトルを潜在的意味解析により200次元に圧縮した場合。
sim_matrix_lsa = cosine_similarity(vectors_lsa[doc_ids], vectors_lsa[doc_ids])
display(pd.DataFrame(sim_matrix_lsa, index=doc_ids, columns=doc_ids))


In [ ]:
# TF-IDF重み付けと次元圧縮をこの順序で適用するためのパイプラインを作成する。
vectorizer_lsa = make_pipeline(vectorizer_tfidf, lsa)


In [ ]:
# 「グルメ レストラン」で次元圧縮された特徴ベクトルで検索を行う。
search_by_query("グルメ レストラン", vectorizer_lsa, vectors_lsa)


In [ ]:
%%time
# LDAで50個のトピックを前提とした潜在的ディリクレ配分法の学習を行う。
lda = LatentDirichletAllocation(n_components=50)
vectors_lda = lda.fit_transform(vectors_tfidf)


In [ ]:
# 変数vectors_ldaの行列の形を表示する。
print(vectors_lda.shape)


In [ ]:
# インデックス3114の文書のトピックの分布の表示。
print(vectors_lda[3114])


In [ ]:
# インデックス3114の文書のトピックの分布の合計値の表示。
print(vectors_lda[3114].sum())


In [ ]:
# インデックス3114の文書がどのようなトピックで構成されているかを表示
for i, v in sorted(enumerate(vectors_lda[3114]), key=lambda x:x[1]):
    if v > 0.0001:
        print(f"トピック{i:2}の重み：{v:.4f}")
max_index = i


In [ ]:
# トピックの情報が格納された行列の形を表示する。
print(lda.components_.shape)


In [ ]:
# トピックmax_indexでどのような語の重みが大きいかを表示する。
topic_id = max_index
# 語のインデックスと重みを対応づける。
elements = zip(lda.components_[topic_id], vocabulary)
elements = sorted(elements, reverse=True)
# 重みが大きい語と重みを表示
for i in range(min(len(elements), 5)):
    print(f"トピック（{topic_id}）での語「{elements[i][1]}」の重みは{elements[i][0]:.3f}")


In [ ]:
# インデックスmax_indexのトピックのワードクラウドの表示。
show_wordcloud_by_vector(lda.components_[topic_id], vocabulary)


In [ ]:
# トピックmax_indexの重みが大きい文書を表示する。
topic_id = max_index
# 全文書におけるトピックmax_indexの重みと、文書のインデックスを対応づける。
elements = zip(vectors_lda[:,topic_id], range(num_docs))
elements = sorted(elements, reverse=True)
# トピックの重みが大きい文書と重みを表示
for i in range(min(len(elements), 35)):
    print(f"文書（{elements[i][1]}）でのトピック（{topic_id}）の重みは{elements[i][0]:.3f}")


In [ ]:
# インデックス3174の文書のTF-IDF重み付けによるワードクラウドを表示する。
show_wordcloud_by_vector(vectors_tfidf[3174], vocabulary)


## 4.2 文書のクラスタリング


In [ ]:
%%time
# 最長距離法でコサイン類似度を用いてクラスタリングを行う。
clustering_agg = AgglomerativeClustering(linkage="complete", affinity="cosine", compute_full_tree=True)
clustering_agg.fit_predict(vectors_lsa)


In [ ]:
# 階層クラスタリングの結果からクラスタを構成する関数の定義。
def get_labels(num_clusters, children):
    num_samples = len(children) + 1
    label_dict = {i: i for i in range(num_samples)}
    inv_label_dict = {i: {i} for i in range(num_samples)}
    num_marge = num_samples - num_clusters
    for cluster, (left, right) in enumerate(children[:num_marge], num_samples):
        members = inv_label_dict.pop(left) | inv_label_dict.pop(right)
        inv_label_dict[cluster] = members
        for member in members:
            label_dict[member] = cluster
    labels = np.zeros(num_samples)
    for cluster, members in enumerate(inv_label_dict.values()):
        labels[list(members)] = cluster
    return labels


In [ ]:
# 20個のクラスタを構成する。
labels = get_labels(20, clustering_agg.children_)


In [ ]:
# 得られた20個のクラスタの情報を表示する。
print(f"文書総数：{len(labels)}")
print(labels)


In [ ]:
# クラスタ数を10から150まで変化させたときの調整ランド指数の変化をみる。
X = range(10, 151, 5)
Y = []
for num_clusters in X:
    labels = get_labels(num_clusters, clustering_agg.children_)
    Y.append(adjusted_rand_score(df["category"], labels))
# クラスタ数と調整ランド指数の折れ線グラフを描く。
plt.plot(X, Y)
# 日本語フォントを取得する。
fp = FontProperties(fname=get_font_path())
plt.xlabel("クラスタ数", fontproperties=fp)
plt.ylabel("調整ランド指数", fontproperties=fp)
plt.show()


In [ ]:
# それぞれのカテゴリーとクラスタに属する文書をまとめる。
# 30個のクラスタにクラスタリングした結果を取得する。
num_clusters = 30
labels = get_labels(num_clusters, clustering_agg.children_)
# それぞれのカテゴリーとクラスタに属する文書をまとめる。
cmt = confusion_matrix(df["category"].apply(category_dirs.index).values, labels)
cmt = cmt[:num_clusters][:len(category_dirs)]
cmt = cmt / num_docs * 100


In [ ]:
# カテゴリーとクラスタの関係を可視化する。
plt.figure(figsize=(16, 8))
sns.heatmap(cmt, annot=True, fmt="1.1f", cmap="Blues")
# 日本語フォントを取得する。
fp = FontProperties(fname=get_font_path())
plt.xlabel("クラスタ", fontproperties=fp)
plt.ylabel("カテゴリー", fontproperties=fp)
plt.show()


In [ ]:
# インデックス29のクラスタのワードクラウドを表示する。
# 【注】ここでインデックス29を選んでいますが、クラスタリングの結果を見ながら適当なクラスタを選択して下さい。
label = 29
doc_ids = np.where(labels==label)[0]
doc_vectors = vectors_tfidf[doc_ids]
mean_vector = np.asarray(doc_vectors.mean(axis=0))[0]
show_wordcloud_by_vector(mean_vector, vocabulary)


In [ ]:
# L2正規化を行う。
vectors_lsa_norm = normalize(vectors_lsa, norm="l2")


In [ ]:
%%time
# K-Means法でクラスタリングを行う。
num_clusters = 9
clustering_kmeans = KMeans(n_clusters=num_clusters)
labels = clustering_kmeans.fit_predict(vectors_lsa_norm)


In [ ]:
# K-Means法の結果の調整ランド指数を表示する。
print(adjusted_rand_score(df["category"], labels))


In [ ]:
# それぞれのカテゴリーとクラスタに属する文書をまとめる。
cmt = confusion_matrix(df["category"].apply(category_dirs.index).values, labels)
cmt = cmt[:num_clusters][:len(category_dirs)]
cmt = cmt / num_docs * 100


In [ ]:
# カテゴリーとクラスタの関係を可視化する。
plt.figure(figsize=(16,8))
sns.heatmap(cmt, annot=True, fmt="1.1f", cmap="Blues")
# 日本語フォントを取得する。
fp = FontProperties(fname=get_font_path())
plt.xlabel("クラスタ", fontproperties=fp)
plt.ylabel("カテゴリー", fontproperties=fp)
plt.show()
